![](ubc_header.png)

# Climate-Friendly Food Systems (CFFS) Labelling Project

### The University of British Columbia

****

## Part I: Data Preprocessing

## Set up and Import Libraries

In [1]:
#pip install -r requirements.txt

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import csv
from itertools import islice
from decimal import Decimal
import xml.etree.ElementTree as et
from xml.etree.ElementTree import parse
import openpyxl
import pytest

In [3]:
# RUN ONLY ONCE
# os.chdir is used to change the current directory to the specified path
os.chdir("../") # Sets path to the repo folder as it is one level above where this file exists!
path = os.getcwd()
print(path)

/Users/vivaanwadhwa/Documents/GitHub/CFFS_sharon_2024


****

## Load Data Files

### Set Data File Path

In [4]:
# Select data file path for the chosen venue and time range where the recipes data stored

# OK 2023
# filepath_list = glob.glob(os.path.join(os.getcwd(), "data", "raw", "OK", "*.oc"))

# Totem 2023
# filepath_list = glob.glob(os.path.join(os.getcwd(), "data", "raw", "Feast", "*.oc"))

# Gather 2023
# filepath_list = glob.glob(os.path.join(os.getcwd(), "data", "raw", "Gather", "*.oc"))

# filepath_list = glob.glob(os.path.join(os.getcwd(), "data", "raw", "BLUEBERRY_FESTIVAL_24"))

filepath_list = glob.glob(os.path.join(os.getcwd(), "data", "raw", "HARVEST_FESTIVAL_24"))

filepath_list

['/Users/vivaanwadhwa/Documents/GitHub/CFFS_sharon_2024/data/raw/HARVEST_FESTIVAL_24']

### Import Items List

In [5]:
# Read items.xml files in the filepath_list and construct a dataframe
ItemId = []
Description = []
CaseQty = []
CaseUOM = []
PakQty = []
PakUOM = []
InventoryGroup = []


# from the items xml file, findtext of CaseQty, CaseUOM, PakQty, PakUOM, and InventoryGroup
# then append it on the lists above

for filepath in filepath_list:
    path = filepath + '/Items.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
        for item in xtree.iterfind('Item'):
            ItemId.append(item.attrib['id'])
            Description.append(item.findtext('Description'))
            CaseQty.append(item.findtext('CaseQty'))
            CaseUOM.append(item.findtext('CaseUOM'))
            PakQty.append(item.findtext('PakQty'))
            PakUOM.append(item.findtext('PakUOM'))
            InventoryGroup.append(item.findtext('InventoryGroup'))

            
# Create a dataframe from the lists created above.
        
Items = pd.DataFrame({'ItemId': ItemId, 'Description': Description, 'CaseQty': CaseQty, 
                      'CaseUOM': CaseUOM, 'PakQty': PakQty, 'PakUOM': PakUOM, 'InventoryGroup': InventoryGroup}
                    )

Items.drop_duplicates(inplace=True)

Items.reset_index(drop=True, inplace=True)

In [6]:
# creates a new array with unique ItemIds
all_id_list = Items["ItemId"].unique()

In [7]:
# None of the items are egg yolk liq
Items.loc[Items["Description"] == "Egg Yolk Liq"]

,ItemId,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup


In [8]:
# Sumplemental option for egg, vegan option
Items.loc[Items["ItemId"] == "I-68700"]

,ItemId,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup


In [9]:
# Gives you the list of breads
breadlist = []


# If the items in the Items list is a "LOAF", "SANDWICH", or "BREAD" then we should add it in the breadlist
# We append it onto the breadlist at position row["ItemId"]
for ind, row in Items.iterrows():
    if ("LOAF" or "SANDWICH" "BREAD") in row["Description"]:
        breadlist.append(row["ItemId"])

breadlist

['I-62929']

In [10]:
Items = Items[~Items["InventoryGroup"].isin(["DISPOSABLES", "SUPPLY"])]

In [11]:
# Based on info below there are 486 rows and 7 columns
Items.shape

(144, 7)

In [12]:
Items.dtypes

ItemId            object
Description       object
CaseQty           object
CaseUOM           object
PakQty            object
PakUOM            object
InventoryGroup    object
dtype: object

In [13]:
# Save the dataframe to csv
path = os.path.join(os.getcwd(), "data", "preprocessed", "Items_List.csv")
Items.to_csv(path, index = False, header = True)

### Import Ingredients List

In [14]:
# Read ingredients.xml files in the filepath_list and construct a dataframe
IngredientId = []
Conversion = []
InvFactor = []
Qty = []
Recipe = []
Uom = []

# Using the Ingredients XML file, we extract attributes containing ingredients, conversion, invFactor, qty, recipe, and uom. 
# Then we append it onto the IngredientId, Coversion, InvFactor, Qty, Recipe, and Uom lists
# Then we create a dataframe using the lists created. 

for filepath in filepath_list:
    path = filepath + '/Ingredients.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
        for x in xtree.iterfind('Ingredient'):
            IngredientId.append(x.attrib['ingredient'])
            Conversion.append(x.attrib['conversion'])
            InvFactor.append(x.attrib['invFactor'])
            Qty.append(x.attrib['qty'])
            Recipe.append(x.attrib['recipe'])
            Uom.append(x.attrib['uom'])
    
Ingredients = pd.DataFrame({'IngredientId': IngredientId, 'Qty': Qty,'Uom': Uom, 'Conversion': Conversion, 
                      'InvFactor': InvFactor,'Recipe': Recipe}).drop_duplicates()
Ingredients.drop_duplicates(subset=["IngredientId", "Recipe"], inplace=True)

Ingredients.reset_index(drop=True, inplace=True)

In [15]:
Ingredients.loc[Ingredients["IngredientId"] == "I-29389"]

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe


In [16]:
# This will output the IngredientId on the right side and the number of times the ingredient appears in the Ingredients
# dataframe on the left side. 

# The duplicated() method returns a Series with True and False values that describe which rows in the DataFrame are 
# duplicated and not.

check = Ingredients["IngredientId"].duplicated()

# The line below tells us at which index is the ingredient duplicated. For example I-4598 exists on index 2 of the dataframe and
# then again on index 8. So duplicate is first true on index 8, which is why it is printed below. It also appears in a below 
# index so it is printed again with that index number. 
Ingredients["IngredientId"][check]

6       I-3660
10      I-3118
14      I-3290
15      I-3658
17      I-6020
        ...   
293     I-6026
298    I-17159
300     I-3389
308     I-3223
323    P-36563
Name: IngredientId, Length: 123, dtype: object

In [17]:
# We can see from the printed items above that I-64877 does not appear because it does not get duplicated
Ingredients.loc[Ingredients["IngredientId"] == "I-64877"]

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe


In [18]:
Ingredients

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
0,I-2979,2.000,Kg,1000.00000000,0.0909,P-10936
1,I-3658,10.000,Kg,1.00000000,0.4545,P-10936
2,I-3660,10.000,Kg,1.00000000,0.4545,P-10936
3,I-3118,1300.000,g,0.00100000,43.9189,P-10951
4,I-3290,20.000,Kg,1.00000000,0.6757,P-10951
...,...,...,...,...,...,...
329,P-29426,1.000,6 pk,1.00000000,1.0000,R-63503
330,P-73969,1.000,ea,1.00000000,1.0000,R-73975
331,P-73970,1.000,srvg,1.00000000,1.0000,R-73975
332,P-73973,1.000,srvg,1.00000000,1.0000,R-73975


In [19]:
# Below we are checking if we can extract an entire recipe. So we can do .loc with the particular recipe id and then 
# print all the data points (ingredientId, Qty, Uom, Conversion, InvFactor) for that recipe. 
Ingredients.loc[Ingredients["Recipe"] == "R-68698"]

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe


In [20]:
Items

,ItemId,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-2853,ANCHOVIES FILLET*,1.700,Kg,1.000,Kg,FOOD - GROCERY
1,I-3118,BAKING POWDER*MAGIC,5.000,Kg,1.000,Kg,FOOD - GROCERY
2,I-3119,BAKING SODA*,5.000,Kg,1000.000,g,FOOD - GROCERY
3,I-4507,BANANA,5.000,lb,2.000,piece,PRODUCE
4,I-3143,BEANS CHICK*PEAS GABANZO CAN,6.000,LG CAN,2.840,L,FOOD - GROCERY
...,...,...,...,...,...,...,...
149,I-6850,YOGURT GREEK*,1.000,TUB,4000.000,g,DAIRY
150,I-1490,ZEST FLAVOURS SUGAR ORANG,6.000,lb,1.000,lb,BAKING-RAW INGREDIENTS
151,I-31129,ZEST LEMON SUGARED REPAK,5.000,lb,1.000,lb,BAKING-RAW INGREDIENTS
152,I-10491,ZUCCHINI,5.000,lb,1.000,lb,PRODUCE


In [21]:
Ingredients.shape

(334, 6)

In [22]:
Ingredients.dtypes

IngredientId    object
Qty             object
Uom             object
Conversion      object
InvFactor       object
Recipe          object
dtype: object

In [23]:
# Save the dataframe to csv
path = os.path.join(os.getcwd(), "data", "preprocessed", "Ingredients_List.csv")
Ingredients.to_csv(path, index = False, header = True)

### Import Preps List

In [24]:
# Read preps.xml files in the filepath_list and construct a dataframe
PrepId = []
Description = []
PakQty = []
PakUOM = []
InventoryGroup = []


# Here we do the same thing for the Preps XML file where we find the columns using attrib function and then append it onto
# the dataframe called Preps. 

for filepath in filepath_list:
    path = filepath + '/Preps.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
        for x in xtree.iterfind('Prep'):
            PrepId.append(x.attrib['id'])
            Description.append(x.findtext('Description'))
            PakQty.append(x.findtext('PakQty'))
            PakUOM.append(x.findtext('PakUOM'))
            InventoryGroup.append(x.findtext('InventoryGroup'))
    
Preps = pd.DataFrame({'PrepId': PrepId, 'Description': Description,
                  'PakQty': PakQty, 'PakUOM':PakUOM, 'InventoryGroup': InventoryGroup}).drop_duplicates()
preps_columns = Preps.columns
Preps.drop_duplicates(subset=["PrepId"], inplace=True)

Preps.reset_index(drop=True, inplace=True)

In [25]:
Preps

,PrepId,Description,PakQty,PakUOM,InventoryGroup
0,P-27444,BAR|Granola|6ea,1.000,6 pk,BAKED GOODS
1,P-33401,BAR|UBC Granola,432.000,ea,
2,P-29428,BS|Muffin|Fruit Medley| 6ea,1.000,6 pk,BAKED GOODS
3,P-29429,BS|Muffin|Seasonal|6ea,1.000,6 pk,BAKED GOODS
4,P-29426,BS|Muffin|Seeded Carrot|6ea,1.000,6 pk,BAKED GOODS
5,P-62742,BS|Muffin|Vegan Choc Cran| 6ea,1.000,6 pk,BAKED GOODS
6,P-29433,BS|Scone|Seasonal|6ea,1.000,6 pk,BAKED GOODS
7,P-10955,CAKE|Ponderosa,160.000,PTN,BAKED GOODS
8,P-27445,CAKE|Ponderosa 6ea,1.000,6 pk,BAKED GOODS
9,P-56813,CHOPPED|Chives*,0.950,Kg,


In [26]:
# This tells us that there are not any duplicates in the Preps dataframe. There is a different process|ingredient pair for 
# all the recipes. 
check = Preps["PrepId"].duplicated().any()
print(check)

False


In [27]:
# There are 546 rows and 5 columns
Preps.shape

(59, 5)

In [28]:
# Here we see there is only one place P-50739 is used in the Preps
Preps.loc[Preps["PrepId"] == "P-50739"]

,PrepId,Description,PakQty,PakUOM,InventoryGroup


In [29]:
Preps.dtypes

PrepId            object
Description       object
PakQty            object
PakUOM            object
InventoryGroup    object
dtype: object

In [30]:
# Save the dataframe to csv
path = os.path.join(os.getcwd(), "data", "preprocessed", "Preps_List.csv")
Preps.to_csv(path, index = False, header = True)

### Import Products List

In [31]:
# Read products.xml files in the filepath_list and construct a dataframe
ProdId = []
Description = []
SalesGroup = []

# From the XML file for products append the id into ProdId, description into Description, and SalesGroup into SalesGroup list.
# Make a dataframe out of the 3 lists
# Then also drop the duplicates in the Products dataframe

for filepath in filepath_list:
    path = filepath + '/Products.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
        for x in xtree.iterfind('Prod'):
            ProdId.append(x.attrib['id'])
            Description.append(x.findtext('Description'))
            SalesGroup.append(x.findtext('SalesGroup'))
        
Products = pd.DataFrame({'ProdId': ProdId, 'Description': Description, 'SalesGroup': SalesGroup})
Products.drop_duplicates(inplace=True)

Products.reset_index(drop=True, inplace=True)

In [32]:
Products

,ProdId,Description,SalesGroup
0,R-73975,HARVEST Feastival 2024,FOOD
1,R-62876,HARVEST|Cake|Granola Bar 6ea,FOOD - BAKERY & DESSERTS
2,R-33249,HARVEST|Cake|Ponderosa 6ea,FOOD - BAKERY & DESSERTS
3,R-63503,HARVEST|Muffin|Carrot (6ea),FOOD - BAKERY & DESSERTS
4,R-33289,HARVEST|Muffin|Fruit Med (6ea),FOOD - BAKERY & DESSERTS
5,R-62998,HARVEST|Muffin|Seasonal (6ea),FOOD - BAKERY & DESSERTS
6,R-62877,HARVEST|Muffin|Vegan (6ea),FOOD - BAKERY & DESSERTS
7,R-62875,HARVEST|Pot|B.berry Chia Parfa,FOOD - BAKERY & DESSERTS
8,R-62874,HARVEST|Pot|Chia Pudding,CK - FOOD
9,R-37215,HARVEST|Pot|Overnight Oat,CK - FOOD


In [33]:
# Here we can see that there is only one R-56966 in the products 
Products.loc[Products["ProdId"] == "R-56966"]

,ProdId,Description,SalesGroup


In [34]:
Products.dtypes

ProdId         object
Description    object
SalesGroup     object
dtype: object

In [35]:
# Here we can see that there is only one R-68698 in the products 
Products.loc[Products["ProdId"] == "R-68698"]

,ProdId,Description,SalesGroup


In [36]:
Products.shape

(20, 3)

In [37]:
Products.dtypes

ProdId         object
Description    object
SalesGroup     object
dtype: object

In [38]:
# Save the dataframe to csv
path = os.path.join(os.getcwd(), "data", "preprocessed", "Products_List.csv")
Products.to_csv(path, index = False, header = True)

### Import Conversions List

In [39]:
# Read conventions.xml files in the filepath_list and construct a dataframe
ConversionId = []
Multiplier = []
ConvertFromQty = []
ConvertFromUom = []
ConvertToQty = []
ConvertToUom = []

# From the XML file for Conversions append the id into ConversionId, multiplier into Multiplier, ConvertFrom->qty into 
# ConvertFromQty,ConvertFrom->uom into ConvertFromUom, ConvertTo->qty into ConvertToQty and and ConvertTo->uom into the
# CovertToUom list. 
# Make a dataframe out of the 3 lists
# Then also drop the duplicates in the Products dataframe


for filepath in filepath_list:
    path = filepath + '/Conversions.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
        for x in xtree.iterfind('Conversion'):
            ConversionId.append(x.attrib['id'])
            Multiplier.append(x.attrib['multiplier'])
            ConvertFromQty.append(x.find('ConvertFrom').attrib['qty'])
            ConvertFromUom.append(x.find('ConvertFrom').attrib['uom'])
            ConvertToQty.append(x.find('ConvertTo').attrib['qty'])
            ConvertToUom.append(x.find('ConvertTo').attrib['uom'])
    
    
Conversions = pd.DataFrame({'ConversionId': ConversionId, 'Multiplier': Multiplier, 'ConvertFromQty': ConvertFromQty,
                           'ConvertFromUom': ConvertFromUom, 'ConvertToQty': ConvertToQty, 'ConvertToUom': ConvertToUom}
                          ).drop_duplicates()

Conversions.reset_index(drop=True, inplace=True)

In [40]:
# Here we can see for example that to convert 1.14 L to 1 L the multiplier is 0.877 since 1/1.14 = 0.877
Conversions

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,,1.00000000,1.0000,XXX,1.0000,L
1,,0.87719298,1.0000,1.14L,1.1400,L
2,,0.66666667,1.0000,1.5L,1.5000,L
3,,0.57142857,1.0000,1.75 L,1.7500,L
4,,0.50000000,1.0000,2L,2.0000,L
...,...,...,...,...,...,...
110,I-8058,0.00102041,1.0000,L,980.0000,g
111,I-10760,0.02777778,1.0000,cup,36.0000,g
112,I-10760,0.02083333,1.0000,leaf,48.0000,g
113,I-10760,0.00200000,1.0000,HEAD,500.0000,g


In [41]:
# Here we can check that there are no ingredients listed since 
Conversions.loc[Conversions["ConversionId"] == "I-4582"]

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom


In [42]:
all_id_list = Items["ItemId"].unique()
all_conv_list = Conversions["ConversionId"].unique()

print("All unique IDs list\n")
print(all_id_list)
print("\n")
print("All unique Conversions list\n")
print(all_conv_list)

All unique IDs list

['I-2853' 'I-3118' 'I-3119' 'I-4507' 'I-3143' 'I-11556' 'I-32246' 'I-4526'
 'I-7185' 'I-4530' 'I-33601' 'I-4531' 'I-4534' 'I-62929' 'I-4538' 'I-6020'
 'I-4555' 'I-4558' 'I-4556' 'I-4572' 'I-4590' 'I-4579' 'I-72465' 'I-2122'
 'I-2126' 'I-2128' 'I-2141' 'I-3993' 'I-3623' 'I-3222' 'I-2979' 'I-40951'
 'I-3223' 'I-1472' 'I-3239' 'I-65801' 'I-4628' 'I-1497' 'I-3272' 'I-63072'
 'I-2156' 'I-2167' 'I-3283' 'I-2986' 'I-3290' 'I-2612' 'I-4657' 'I-9685'
 'I-3299' 'I-67010' 'I-23753' 'I-4508' 'I-4617' 'I-4620' 'I-4695' 'I-4724'
 'I-4772' 'I-4983' 'I-4919' 'I-3321' 'I-1813' 'I-6443' 'I-1759' 'I-13254'
 'I-52598' 'I-11555' 'I-4844' 'I-10760' 'I-3342' 'I-60772' 'I-2236'
 'I-8058' 'I-11341' 'I-57771' 'I-11345' 'I-62801' 'I-3369' 'I-3370'
 'I-3702' 'I-8077' 'I-3379' 'I-3182' 'I-3006' 'I-56382' 'I-3387' 'I-3389'
 'I-3390' 'I-3392' 'I-29519' 'I-4757' 'I-4750' 'I-57772' 'I-71841'
 'I-3245' 'I-3431' 'I-19567' 'I-23341' 'I-6812' 'I-3642' 'I-3581' 'I-1119'
 'I-72219' 'I-72220' 'I-4821' 'I

In [43]:
# Here we have the number of items in the list that are a part of the "all_id_list" but not part of the "all_conv_list"
missing_conv_id = []

for item in all_id_list:
    if item not in all_conv_list:
        missing_conv_id.append(item)
        
missing_conv_id
print(len(missing_conv_id))

104


In [44]:
Conversions.shape

(115, 6)

In [45]:
Conversions.loc[Conversions["ConversionId"] == "I-29389"]

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom


In [46]:
Conversions.dtypes

ConversionId      object
Multiplier        object
ConvertFromQty    object
ConvertFromUom    object
ConvertToQty      object
ConvertToUom      object
dtype: object

In [47]:
# Save the dataframe to csv
path = os.path.join(os.getcwd(), "data", "preprocessed", "Conversions_List.csv")
Conversions.to_csv(path, index = False, header = True)

***
## Data Summary

In [48]:
# Summary of raw data imported for evaluation
# Here we have a summary of the number of items, preps, ingredients, products, conversions

datasum = pd.DataFrame([Items.shape, Preps.shape, Ingredients.shape, Products.shape, Conversions.shape],
                       columns = ['count', 'columns'], 
                       index = ['Items', 'Preps', 'Ingredients', 'Products', 'Conversions'])
datasum

,count,columns
Items,144,7
Preps,59,5
Ingredients,334,6
Products,20,3
Conversions,115,6
